<a href="https://colab.research.google.com/github/KhaledRobot/ML/blob/main/SuccessfulLLMex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install openai

In [2]:
pip install --upgrade openai

In [3]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 4.7 MB/s eta 0:00:00


In [7]:
import pandas as pd
import re
import csv
import torch
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from groq import Groq, RateLimitError
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import random

# Initialize GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.max_split_size_mb = 128

# ==== FULL SYSTEM PROMPTS ====
DEFENSE_SYSTEM = """You are an experienced defense attorney. Your role is to defend your client against the charges by:
1. Challenging the prosecution's evidence
2. Highlighting reasonable doubt
3. Protecting your client's rights
4. Providing alternative explanations when possible
5. Being persuasive but ethical

You specialize in criminal law and know courtroom procedures perfectly."""
PROSECUTION_SYSTEM = """You are a seasoned prosecutor. Your duties include:
1. Presenting the case against the defendant clearly
2. Establishing guilt beyond reasonable doubt
3. Countering defense arguments effectively
4. Ensuring justice is served
5. Maintaining professional ethics

You're an expert in criminal law and courtroom tactics."""
PLAINTIFF_SYSTEM = """You are the plaintiff in this case. Your role is to:
1. Describe what happened truthfully
2. Explain how you were harmed
3. Answer questions clearly without speculation
4. Stick to facts you personally witnessed
5. Avoid emotional outbursts"""
WITNESS_SYSTEM = """You are an eyewitness testifying in court. Your responsibilities:
1. Answer only what you directly observed
2. Be precise about what you saw/heard
3. Admit when you don't remember
4. Don't speculate or guess
5. Remain calm under questioning"""
DEFENDANT_SYSTEM = """You are the defendant in this criminal case. When speaking:
1. Maintain your presumption of innocence
2. Answer truthfully but carefully
3. Consult with your attorney when needed
4. Don't volunteer unnecessary information
5. Behave respectfully in court"""
JUDGE_SYSTEM = """You are the presiding judge. Your duties:
1. Ensure proper courtroom procedure
2. Remain completely impartial
3. Evaluate evidence objectively
4. Apply the law correctly
5. Deliver a reasoned verdict

You have 20 years of judicial experience and deep legal knowledge."""
# Initialize models
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased").to(device)
sentence_model = SentenceTransformer('all-mpnet-base-v2').to(device)

# Configure Groq client with retry logic
class GroqClientWithRetry:
    def __init__(self, api_key):
        self.client = Groq(api_key=api_key)
        self.last_call_time = 0

    def create_chat_completion(self, **kwargs):
        current_time = time.time()
        elapsed = current_time - self.last_call_time
        min_delay = 1.5  # Minimum delay between calls (1.5s)

        if elapsed < min_delay:
            sleep_time = min_delay - elapsed + random.uniform(0, 0.5)
            time.sleep(sleep_time)

        attempts = 0
        max_attempts = 5
        base_delay = 1.5

        while attempts < max_attempts:
            try:
                self.last_call_time = time.time()
                return self.client.chat.completions.create(**kwargs)
            except RateLimitError as e:
                attempts += 1
                if attempts == max_attempts:
                    raise

                # Exponential backoff with jitter
                delay = base_delay * (2 ** attempts) + random.uniform(0, 1)
                time.sleep(delay)
            except Exception as e:
                raise e

groq_client = GroqClientWithRetry("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")

# Load data
df = pd.read_csv("cases.csv", usecols=["id", "text"])
cases = df.to_dict('records')

# GPU-accelerated processing
def gpu_process_text(text):
    inputs = tokenizer(text[:10000], return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

# Case processing with robust error handling
def process_case(case):
    try:
        # GPU processing
        with torch.cuda.amp.autocast():
            embedding = gpu_process_text(case['text'])
            case['embedding'] = embedding.cpu().numpy()

        # Summarization with retry
        summary = groq_client.create_chat_completion(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
                {"role": "system", "content": "Provide a comprehensive legal case summary..."},
                {"role": "user", "content": case['text'][:15000]}
            ],
            temperature=0.3,
            max_tokens=400
        ).choices[0].message.content.strip()
        case['summary'] = summary

        # Trial simulation
        agents = initialize_agents()
        verdict = run_trial_simulation(agents, summary)

        label = 1 if "not guilty" in verdict.lower() else 0
        return (case['id'], label, "success")

    except RateLimitError as e:
        return (case['id'], -1, f"rate_limit_error: {str(e)}")
    except Exception as e:
        return (case['id'], -1, f"processing_error: {str(e)}")

# Initialize agents with rate limit awareness
class LawyerAgent:
    def __init__(self, name, system_prompt):
        self.name = name
        self.system_prompt = system_prompt
        self.history = []

    def respond(self, prompt, client):
        messages = [
            {"role": "system", "content": self.system_prompt},
            *self.history[-4:],
            {"role": "user", "content": prompt}
        ]
        response = client.create_chat_completion(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=messages,
            temperature=0.7,
            max_tokens=256
        )
        reply = response.choices[0].message.content.strip()
        self.history.extend([
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": reply}
        ])
        return reply

def run_trial_simulation(agents, case_summary):
    transcript = []
    client = GroqClientWithRetry("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")

    # Opening statements
    prosecution_open = agents["prosecution"].respond(f"Opening statement. Case: {case_summary}", client)
    defense_open = agents["defense"].respond("Opening statement responding to prosecution", client)
    transcript.extend([prosecution_open, defense_open])
    time.sleep(1.5)  # Rate limit buffer

    # Examination
    prose_q = agents["prosecution"].respond("Question the witness", client)
    witness_a = agents["witness"].respond(prose_q, client)
    transcript.extend([prose_q, witness_a])
    time.sleep(1.5)

    # Cross-examination
    defense_q = agents["defense"].respond("Cross-examine the witness", client)
    witness_a2 = agents["witness"].respond(defense_q, client)
    transcript.extend([defense_q, witness_a2])
    time.sleep(1.5)

    # Closing arguments
    prose_close = agents["prosecution"].respond("Closing argument", client)
    defense_close = agents["defense"].respond("Closing argument", client)
    transcript.extend([prose_close, defense_close])
    time.sleep(1.5)

    # Verdict
    verdict = agents["judge"].respond(
        f"Based on this trial:\n{case_summary[:5000]}\n\n"
        "Key exchanges:\n" + "\n".join(transcript[-4:]) +
        "\n\nRender verdict ('guilty' or 'not guilty') with reasoning.",
        client
    )
    return verdict

# Main processing with enhanced rate control
def process_cases_parallel(cases, max_workers=3, cases_per_batch=5):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []

        for i in range(0, min(50, len(cases)), cases_per_batch):
            batch = cases[i:i + cases_per_batch]

            # Submit batch with delay between submissions
            for case in batch:
                futures.append(executor.submit(process_case, case))
                time.sleep(0.5)  # Stagger submissions

            # Monitor progress
            print(f"Submitted batch {i//cases_per_batch + 1}, GPU Mem: {torch.cuda.memory_allocated()/1024**3:.2f}GB")

        # Process results as they complete
        for future in as_completed(futures):
            case_id, label, status = future.result()
            results.append((case_id, label, status))

            # Update monitoring
            if "error" in status:
                print(f"Case {case_id} failed: {status}")
            else:
                print(f"Case {case_id} completed successfully")

    return results

# Run processing
with open("verdicts.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id", "label", "status"])

    results = process_cases_parallel(cases)
    for case_id, label, status in results:
        writer.writerow([case_id, label, status])
        f.flush()

torch.cuda.empty_cache()
print("Processing complete. Results saved to verdicts.csv")

<ipython-input-7-80b05206739e>:115: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Submitted batch 1, GPU Mem: 1.68GB
Submitted batch 2, GPU Mem: 1.68GB
Submitted batch 3, GPU Mem: 1.68GB
Submitted batch 4, GPU Mem: 1.68GB
Submitted batch 5, GPU Mem: 1.68GB
Submitted batch 6, GPU Mem: 1.68GB
Submitted batch 7, GPU Mem: 1.68GB
Submitted batch 8, GPU Mem: 1.68GB
Submitted batch 9, GPU Mem: 1.68GB
Submitted batch 10, GPU Mem: 1.68GB
Case 1989_75 completed successfully
Case 1959_102 completed successfully
Case 1979_76 completed successfully
Case 1984_73 completed successfully
Case 1985_233 completed successfully
Case 1991_124 completed successfully
Case 1963_191 completed successfully
Case 1992_1 completed successfully
Case 1963_285 completed successfully
Case 1961_166 completed successfully
Case 1990_342 completed successfully
Case 1975_183 completed successfully
Case 1962_237 completed successfully
Case 1985_23 completed successfully
Case 1971_570 completed successfully
Case 1984_63 completed successfully
Case 1988_248 completed successfully
Case 1988_201 completed suc